In [1]:
from lxml import etree, html
from lxml.html.clean import clean_html
import pandas as pd
from datetime import date, datetime
import requests
import re
import os
import shutil
from ASECORP_BBDD import tagea_BBDD_ASECORP
from selenium import webdriver
import time

# Mueve todos los fichero del directorio de trabajo a otro de archivo

# Recoge resumen diario del BOE de hoy

In [2]:

today = date.today()

# dd/mm/YYYY
hoy = today.strftime("%Y%m%d")
print("Fecha de Hoy =", hoy)

# dd
d = today.strftime("%d")
print("dia =", d)

# mm
m = today.strftime("%m")
print("mes =", m)

# YYYY
Y = today.strftime("%Y")
print("año =", Y)

print(today.strftime("%d/%m/%Y"))

Fecha de Hoy = 20210304
dia = 04
mes = 03
año = 2021
04/03/2021


In [3]:
def save_html(html, path):
    with open(path, 'wb') as f:
        f.write(html)

In [4]:
# Define expresiones REGEX para búsqueda de leyes, decretos, etc. referenciadas anteriormente
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

# ------------------------------------------------------------------------

# DOGC

In [5]:
URL_HTML_resumen = "https://dogc.gencat.cat/es/inici/"

In [6]:
### Necesita libreria Selenium para renderizar JS script

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_resumen)
#print(driver.page_source)
response = driver.page_source

sumario_HTML = html.fromstring(response)
#print(sumario_HTML.xpath('//h2[@class="titol-component"]/a/@href'))
numero_diario = sumario_HTML.xpath('//h2[@class="titol-component"]/a/text()')
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
secciones_link = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
secciones_linkHTML = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]
# numero_pagina = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[3]/td[4]/text()')
# NID = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[2]/td[2]/text()')
# rango = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[5]/td[2]/text()')
# organo_emisor = sumario_HTML.xpath('//table[@class="tablaDetalle"]/tbody/tr[7]/td[2]/text()')
# 
# print(numero_diario[0].strip(), numero_pagina[0].strip(), NID[0].strip(), rango[0].strip(), organo_emisor[0].strip())
# print()
print(numero_diario)
print(secciones)
print(secciones_linkHTML)

driver.quit()

['DOGC núm. 8356']
['Disposiciones', 'Cargos y personal', 'Anuncios de la Generalidad de Cataluña', 'Anuncios de la Administración local', 'Anuncios de la Administración de justicia ']
['https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Disposiciones', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Cargos_y_personal', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Generalidad_de_Cataluña', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Administración_local', 'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Administración_de_justicia_']


In [7]:
#response

## Recoge Nombre Secciones Sumario

In [8]:
secciones = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/text()')
#print(secciones)
df_secciones_sumarios = pd.DataFrame()
for seccion in secciones:
    df_secciones_sumarios = df_secciones_sumarios.append({'Seccion': seccion}, ignore_index=True)


In [9]:
df_secciones_sumarios

,Seccion
0,Disposiciones
1,Cargos y personal
2,Anuncios de la Generalidad de Cataluña
3,Anuncios de la Administración local
4,Anuncios de la Administración de justicia


## Recoge Valores para formar URLs Secciones Sumario

In [10]:
URL_base_sumario = sumario_HTML.xpath('//div[@class="llistat_cont"]/ul/li/a/@href')
URL_sumario = [ ('https://dogc.gencat.cat' + item) for item in secciones_link ]

df_URL_sumarios = pd.DataFrame()

for URL_seccion in URL_base_sumario:    
    URL_sumario = 'https://dogc.gencat.cat' + str(URL_seccion)
    print(URL_sumario)
    df_URL_sumarios = df_URL_sumarios.append({'URL_Seccion': URL_sumario}, ignore_index=True)

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Disposiciones
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Cargos_y_personal
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Generalidad_de_Cataluña
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Administración_local
https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Anuncios_de_la_Administración_de_justicia_


In [11]:
df_URL_sumarios

,URL_Seccion
0,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [12]:
 df_sumarios = pd.concat([df_secciones_sumarios, df_URL_sumarios], axis=1)

In [13]:
df_sumarios

,Seccion,URL_Seccion
0,Disposiciones,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
1,Cargos y personal,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
2,Anuncios de la Generalidad de Cataluña,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
3,Anuncios de la Administración local,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...
4,Anuncios de la Administración de justicia,https://dogc.gencat.cat/es/sumari-del-dogc/?nu...


In [15]:
df_sumarios['URL_Seccion'][0]

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Disposiciones'

## Recoge Items en Seccion Disposiciones

In [16]:
URL_HTML_disposiciones = df_sumarios[df_sumarios['Seccion'] == 'Disposiciones']

In [17]:
print(URL_HTML_disposiciones['Seccion'][0])

Disposiciones


In [18]:
print(URL_HTML_disposiciones['URL_Seccion'][0])

https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Disposiciones


In [19]:
URL_HTML_disposiciones = URL_HTML_disposiciones['URL_Seccion'][0]

In [20]:
URL_HTML_disposiciones

'https://dogc.gencat.cat/es/sumari-del-dogc/?numDOGC=8356#section-Disposiciones'

In [21]:
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get(URL_HTML_disposiciones)
#print(driver.page_source)
response = driver.page_source
### Recoge Items en Seccion Disposiciones
disposiciones_HTML = html.fromstring(response)

#print(html.tostring(disposiciones_HTML))
#print('____________________')

departamentos = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/h3/text()') # Sólo Sección disposiciones
disposiciones = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/text()')
disposiciones_link = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/a/@href')
disposiciones_pdf = disposiciones_HTML.xpath('/descendant::*[@class="wrapper-disposicions"][1]/ul/descendant::li/div/div/a/@href')

# for i in range(len(departamentos)):
#     print(departamentos[i])
#     i += 1
#     #print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[' + str(i) + ']//li/div/a/text()'))
#     print(disposiciones_HTML.xpath('//*[@class="wrapper-disposicions"]/ul[1]//li/div/a/text()'))

print(departamentos)
print(disposiciones)
print(disposiciones_link)
print(disposiciones_pdf)

driver.quit()


['Departamento de la Presidencia', 'Departamento de la Vicepresidencia y de Economía y Hacienda', 'Departamento de Interior', 'Departamento de Educación', 'Departamento de Salud', 'Departamento de Territorio y Sostenibilidad', 'Departamento de Cultura', 'Departamento de Empresa y Conocimiento', 'Departamento de Políticas Digitales y Administración Pública']
['CORRECCIÓN DE ERRATAS en la Ley 9/2019, de 23 de diciembre, de modificación de la Ley 16/2017, del cambio climático, en cuanto al impuesto sobre las emisiones de dióxido de carbono de los vehículos de tracción mecánica (DOGC núm. 8032, de 30.12.2019).', 'ORDEN VEH/49/2021, de 1 de marzo, por la que se aprueban los criterios de ocupación de los inmuebles de uso administrativo de la Generalidad de Cataluña.', 'RESOLUCIÓN INT/570/2021, de 26 de febrero, de modificación de la Resolución INT/3444/2020, de 1 de septiembre, de concesión de medallas de bronce al mérito policial con distintivo azul a varios miembros de cuerpos policiales.'

In [22]:
len(departamentos)

9

In [23]:
DOGC_sumarios = pd.DataFrame(columns=['item_Title','item_urlHTML','item_urlPDF'])

In [24]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF


In [25]:
lista = []
for disposicion in disposiciones:
    lista.append(disposicion)

DOGC_sumarios['item_Title'] = lista

lista = []   
for link_HTML in disposiciones_link:
    #print(link)
    lista.append('https://dogc.gencat.cat' + str(link_HTML))

DOGC_sumarios['item_urlHTML'] = lista

lista = []   
for link_PDF in disposiciones_pdf:
    #print(link)
    lista.append(link_PDF)

DOGC_sumarios['item_urlPDF'] = lista

In [26]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF
0,"CORRECCIÓN DE ERRATAS en la Ley 9/2019, de 23 ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
1,"ORDEN VEH/49/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
2,"RESOLUCIÓN INT/570/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
3,"RESOLUCIÓN INT/571/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
4,"DECRETO 12/2021, de 2 de marzo, de la Inspecci...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
5,"RESOLUCIÓN EDU/572/2021, de 25 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
6,"ORDEN SLT/48/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
7,"EDICTO de 25 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
8,"ACUERDO GOV/22/2021, de 2 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...
9,"RESOLUCIÓN CLT/497/2021, de 16 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...


In [27]:
DOGC_sumarios['item_urlHTML'][1]

'https://dogc.gencat.cat/es/document-del-dogc/?documentId=894795'

In [28]:
### Recoge información de página de detalle con Selenium
### es necesario ya que la página se genera con un JS y
### hay que renderizarla con un headless web browser

options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

DOGC_sumarios['item_fullText'] = ''
DOGC_sumarios['item_urlXML'] = ''
DOGC_sumarios['tipo_doc'] = ''
DOGC_sumarios['fecha_doc'] = ''
DOGC_sumarios['numero_doc'] = ''
DOGC_sumarios['numero_control'] = ''
DOGC_sumarios['entidad_emisora'] = ''
DOGC_sumarios['CVE'] = ''
DOGC_sumarios['numero_DOGC'] = ''
DOGC_sumarios['fecha_DOGC'] = ''
DOGC_sumarios['seccion_DOGC'] = ''

for i, row in DOGC_sumarios.iterrows():
    # carga página HTML y genera árbol
    driver.get(row['item_urlHTML'])
    #print(driver.page_source)

    # clica sobre desplegable 'datos del documento' para generar contenido
    driver.find_element_by_xpath('//div[@class="panel-title"]/a[@href="#disposicions"]').click()
    time.sleep(3)

    response = driver.page_source
    sumario_HTML = html.fromstring(response)

    # Genera columnas de la tabla con Datos de los documentos
    item_urlXML = sumario_HTML.xpath('//*[@id="download"]/a[4]/@href')

    tipo_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Tipo de documento")]/text()[2]')
    fecha_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del documento")]/text()[2]')
    numero_doc = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de documento")]/text()[2]')
    numero_control = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número de control")]/text()[2]')
    entidad_emisora = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Entidad emisora")]/text()[2]')
    CVE = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"CVE")]/text()[2]')
    numero_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Número del DOGC")]/text()[2]')
    fecha_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Fecha del DOGC")]/text()[2]')
    seccion_DOGC = sumario_HTML.xpath('//*[@id="disposicions_cos_bloc"]/li[contains(text()[1],"Sección del DOGC")]/text()[2]')

    DOGC_sumarios['item_urlXML'][i] = item_urlXML

    DOGC_sumarios['tipo_doc'][i] = tipo_doc
    DOGC_sumarios['fecha_doc'][i] = fecha_doc
    DOGC_sumarios['numero_doc'][i] = numero_doc
    DOGC_sumarios['numero_control'][i] = numero_control
    DOGC_sumarios['entidad_emisora'][i] = entidad_emisora
    DOGC_sumarios['CVE'][i] = CVE
    DOGC_sumarios['numero_DOGC'][i] = numero_DOGC
    DOGC_sumarios['fecha_DOGC'][i] = fecha_DOGC
    DOGC_sumarios['seccion_DOGC'][i] = seccion_DOGC

    # Recoge en una lista todos los parágrafos del texto completo
    fullText = []
    for div in sumario_HTML.xpath('//*[@id="fullText"]/div/div/div'):
        fullText.append(div.text_content())

    # Convierte la lista en una string
    fullText = ' '.join(fullText)

    DOGC_sumarios['item_fullText'][i] = fullText


driver.quit()

In [29]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC
0,"CORRECCIÓN DE ERRATAS en la Ley 9/2019, de 23 ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Habiéndose detectado una errata en la Ley 9/20...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Corrección de erratas],[01/03/2021],[],[21061054],[Departamento de la Presidencia],[CVE-DOGC-B-21061054-2021],[8356],[04/03/2021],[Disposiciones]
1,"ORDEN VEH/49/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 159.1.a) del Estatuto de autonomía...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Orden],[01/03/2021],[VEH/0049/2021],[21061055],[Departamento de la Vicepresidencia y de Econo...,[CVE-DOGC-B-21061055-2021],[8356],[04/03/2021],[Disposiciones]
2,"RESOLUCIÓN INT/570/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Interior, mediante la Resol...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[26/02/2021],[INT/0570/2021],[21061058],[Departamento de Interior],[CVE-DOGC-B-21061058-2021],[8356],[04/03/2021],[Disposiciones]
3,"RESOLUCIÓN INT/571/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"\nEl Departamento de Interior, mediante la Re...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[26/02/2021],[INT/0571/2021],[21061059],[Departamento de Interior],[CVE-DOGC-B-21061059-2021],[8356],[04/03/2021],[Disposiciones]
4,"DECRETO 12/2021, de 2 de marzo, de la Inspecci...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 131.2 del Estatuto de autonomía de...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Decreto],[02/03/2021],[012/2021],[21061108],[Departamento de Educación],[CVE-DOGC-B-21061108-2021],[8356],[04/03/2021],[Disposiciones]
5,"RESOLUCIÓN EDU/572/2021, de 25 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución EDU/772/2020, de 24 de ...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[25/02/2021],[EDU/0572/2021],[21061060],[Departamento de Educación],[CVE-DOGC-B-21061060-2021],[8356],[04/03/2021],[Disposiciones]
6,"ORDEN SLT/48/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Decreto 215/1990, de 30 de julio, de promoc...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Orden],[01/03/2021],[SLT/0048/2021],[21061061],[Departamento de Salud],[CVE-DOGC-B-21061061-2021],[8356],[04/03/2021],[Disposiciones]
7,"EDICTO de 25 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de El Cam...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[25/02/2021],[],[21056014],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21056014-2021],[8356],[04/03/2021],[Disposiciones]
8,"ACUERDO GOV/22/2021, de 2 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Instituto Ramon Muntaner, Fundación Privada...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Acuerdo del gobierno ],[02/03/2021],[GOV/0022/2021],[21062071],[Departamento de Cultura],[CVE-DOGC-B-21062071-2021],[8356],[04/03/2021],[Disposiciones]
9,"RESOLUCIÓN CLT/497/2021, de 16 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Visto el Acuerdo del Consejo de Administración...,[https://portaldogc.gencat.ca

In [30]:
DOGC_sumarios['item_fullText'][0]

'Habiéndose detectado una errata en la Ley 9/2019 (publicada en el DOGC núm. 8032, de fecha 30.12.2019), procede realizar las siguientes correcciones:\n\xa0\nEn el artículo 8\nDonde dice:\n«Se modifica el artículo 47 de la Ley 16/2017, de 1 de agosto, del cambio climático, que queda redactado del siguiente modo:\n«Artículo 47. Padrón\n»[…]\n»7. No deben incluirse en el padrón definitivo:\n»[…]\n»b) Los vehículos que devenguen una cuota líquida a la cuantía que se apruebe de conformidad con lo dispuesto por el artículo 26.4 del texto refundido de la Ley de finanzas públicas de Cataluña, aprobado por el Decreto legislativo 3/2002, de 24 de diciembre. Esta regla de no inclusión no se aplica respecto a los vehículos que son de la titularidad de un mismo sujeto pasivo, siempre que la suma de las cuotas líquidas devengadas por todos los vehículos supere dicha cuantía.»\n Debe decir:\n«Se modifica el artículo 47 de la Ley 16/2017, de 1 de agosto, del cambio climático, que queda redactado del 

In [31]:
# Crea nueva columna vacía de tipo lista en tabla_analisis
#DOCM_sumarios['Referencias_completas'] = [[] for i in range(len(tabla_analisis))]
DOGC_sumarios['Tags'] = [[] for i in range(len(DOGC_sumarios))]
DOGC_sumarios['Match_ASECORP_BBDD'] = [[] for i in range(len(DOGC_sumarios))]

In [32]:
pattern = ['Ley [0-9]+\/[0-9]+','Real Decreto [0-9]+\/[0-9]+','Real Decreto Legislativo [0-9]+\/[0-9]+','Real Decreto-ley [0-9]+\/[0-9]+','Orden [A-Z]+\/[0-9]+\/[0-9]+','Orden Circular [0-9]+\/[0-9]+','Reglamento \(UE\) [0-9]+\/[0-9]+', 'Reglamento de Ejeución \(UE\) [0-9]+\/[0-9]+' ,'Sentencia de [0-9]+ de [a-z]+ de [0-9]+','Sentencia [0-9]+\/[0-9]+','Orden de [0-9]+ de [a-z]+ de [0-9]+', 'Resolución de [0-9]+ de [a-z]+ de [0-9]+','Resolución [a-z]+\/[0-9]+\/[0-9]+', 'Nota de Servicio [0-9]+\/[0-9]+', 'Acuerdo multilateral M\-[0-9]+', 'Circular [0-9]+\/[0-9]+', 'Decisión \(UE\) [0-9]+\/[0-9]+', 'Decisión de Ejecución \(UE\) [0-9]+\/[0-9]+']

In [33]:
for i, row in DOGC_sumarios.iterrows():
    # Busca expresiones REGX coincidentes con Patrones definidos
    DOGC_sumarios['Tags'][i] = re.findall('|'.join(pattern), str(row['item_fullText']), flags=re.IGNORECASE)
    print(DOGC_sumarios['Tags'][i])

['Ley 9/2019', 'Ley 16/2017', 'Ley 16/2017']
['Ley 39/2015', 'Ley 19/2014', 'Ley 13/2008', 'Ley 4/1985', 'Ley 13/1989', 'Ley 13/2005', 'Orden de 3 de julio de 1998']
['Resolución INT/3444/2020', 'Ley 39/2015', 'Resolución INT/3444/2020']
['Resolución INT/3439/2020', 'Ley 39/2015', 'Resolución INT/3439/2020']
['Ley 12/2009', 'Orden ENS/289/2002', 'Ley 19/2014', 'Ley 39/2015', 'Ley 26/2010', 'Ley 19/2014', 'Ley 13/2008', 'Ley 13/2008', 'Ley 12/2009', 'Orden ENS/289/2002', 'Ley 12/2009']
['Resolución EDU/772/2020', 'Resolución EDU/772/2020', 'Ley 26/2010', 'Ley 39/2015']
['Orden de 15 de mayo de 1991', 'Ley 15/1990', 'Orden PRE/194/2020', 'Ley 13/2008', 'Orden PRE/194/2020']
['Ley 3/2012', 'Ley 39/2015', 'Ley 29/1998', 'Ley 29/1998', 'Ley 3/2012', 'Ley 2/2007', 'Ley 3/2012']
['Ley 4/2008', 'Ley 5/2017', 'Ley 5/2017', 'Ley 13/2008']
['Resolución CLT/3216/2020', 'Resolución CLT/198/2021', 'Resolución CLT/845/2019', 'Resolución CLT/3216/2020', 'Resolución CLT/198/2021', 'Resolución CLT/845/2

In [34]:
# Elimina Tags duplicados
for i, row in DOGC_sumarios.iterrows():
    DOGC_sumarios['Tags'][i] = list(set(DOGC_sumarios['Tags'][i]))
    print(DOGC_sumarios['Tags'][i])

['Ley 9/2019', 'Ley 16/2017']
['Ley 13/2008', 'Ley 19/2014', 'Ley 13/2005', 'Ley 39/2015', 'Ley 13/1989', 'Orden de 3 de julio de 1998', 'Ley 4/1985']
['Ley 39/2015', 'Resolución INT/3444/2020']
['Ley 39/2015', 'Resolución INT/3439/2020']
['Ley 13/2008', 'Ley 26/2010', 'Ley 12/2009', 'Ley 39/2015', 'Ley 19/2014', 'Orden ENS/289/2002']
['Ley 26/2010', 'Resolución EDU/772/2020', 'Ley 39/2015']
['Orden de 15 de mayo de 1991', 'Orden PRE/194/2020', 'Ley 15/1990', 'Ley 13/2008']
['Ley 29/1998', 'Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012']
['Ley 13/2008', 'Ley 5/2017', 'Ley 4/2008']
['Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/845/2019', 'Orden CLT/4/2013']
['Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/1390/2019', 'Orden CLT/4/2013']
['Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/1390/2019', 'Orden CLT/4/2013']
['Resolución CLT/547/2015', 'Ley 38/2003', '

In [35]:
regex_result = []
[ regex_result.append(tag) for tags in DOGC_sumarios['Tags'] for tag in tags ]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [36]:
print(regex_result)

['Ley 9/2019', 'Ley 16/2017', 'Ley 13/2008', 'Ley 19/2014', 'Ley 13/2005', 'Ley 39/2015', 'Ley 13/1989', 'Orden de 3 de julio de 1998', 'Ley 4/1985', 'Ley 39/2015', 'Resolución INT/3444/2020', 'Ley 39/2015', 'Resolución INT/3439/2020', 'Ley 13/2008', 'Ley 26/2010', 'Ley 12/2009', 'Ley 39/2015', 'Ley 19/2014', 'Orden ENS/289/2002', 'Ley 26/2010', 'Resolución EDU/772/2020', 'Ley 39/2015', 'Orden de 15 de mayo de 1991', 'Orden PRE/194/2020', 'Ley 15/1990', 'Ley 13/2008', 'Ley 29/1998', 'Ley 2/2007', 'Ley 39/2015', 'Ley 3/2012', 'Ley 13/2008', 'Ley 5/2017', 'Ley 4/2008', 'Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/845/2019', 'Orden CLT/4/2013', 'Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/1390/2019', 'Orden CLT/4/2013', 'Orden CLT/47/2014', 'Resolución CLT/198/2021', 'Resolución CLT/3216/2020', 'Resolución CLT/1390/2019', 'Orden CLT/4/2013', 'Resolución CLT/547/2015', 'Ley 38/2003', 'Resolución C

In [37]:
## Elimina duplicados
boletin_flat_list = list(set(regex_result))

In [38]:
boletin_flat_list

['Resolución EDU/772/2020',
 'Orden PRE/18/2016',
 'Resolución CLT/3216/2020',
 'Ley 12/2009',
 'Ley 39/2015',
 'Resolución CLT/547/2015',
 'Ley 4/1985',
 'Ley 17/2015',
 'Ley 15/1990',
 'Real decreto 887/2006',
 'Ley 3/1991',
 'Resolución CLT/187/2021',
 'Ley 29/1998',
 'Ley 4/2008',
 'Ley 13/1989',
 'Resolución INT/3439/2020',
 'Resolución CLT/845/2019',
 'Orden ENS/289/2002',
 'Ley 13/2008',
 'Resolución CLT/198/2021',
 'Ley 26/2010',
 'Resolución CLT/1390/2019',
 'Orden EMO/215/2011',
 'Ley 3/2012',
 'Ley 5/2017',
 'Ley 7/2001',
 'Ley 7/2005',
 'Resolución UNI/73/2003',
 'Orden de 15 de mayo de 1991',
 'Orden de 3 de julio de 1998',
 'Ley 13/2005',
 'Orden CLT/4/2013',
 'Resolución INT/3444/2020',
 'Orden CLT/47/2014',
 'Ley 2/2007',
 'Ley 9/2019',
 'Orden PRE/194/2020',
 'Resolución CLT/3142/2020',
 'Real decreto 1224/2009',
 'Real decreto legislativo 2/2015',
 'Ley 16/2017',
 'Ley 38/2003',
 'Ley 19/2014',
 'Resolución EMC/955/2017']

In [39]:
# ## Importa BBDD ASECORP

# Inicializa datos de BBDD_ASECORP
boletin_ASECORP_flat_list = []
ASECORP_BBDD = pd.DataFrame()
ambitos = []

# Incluir en llamada a función el ambito territorial como lista
# si no se especifica ámbito los incluye todos
boletin_ASECORP_flat_list, ASECORP_BBDD, ambitos = tagea_BBDD_ASECORP(['España','Europa','Cataluña'])

print(ambitos)


['España', 'Europa', 'Cataluña']


In [40]:
#boletin_ASECORP_flat_list

In [41]:
## Busca coincidencias entre lista boletines BOEs explorados y lista boletines de BBDD ASECORP
set(boletin_flat_list) & set(boletin_ASECORP_flat_list)

set()

In [42]:
#DOCM_sumarios['Tags'].isin(ASECORP_BBDD_BOE['Tags'])
for row_to_compare in DOGC_sumarios['Tags']:
    for row_comparing in ASECORP_BBDD['Tags']:
        if set(row_comparing) & set(row_to_compare):
            print(set(row_comparing) & set(row_to_compare))

{'Ley 16/2017'}
{'Ley 9/2019', 'Ley 16/2017'}
{'Ley 13/1989'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 3/2012'}
{'Ley 5/2017'}
{'Ley 5/2017'}
{'Ley 17/2015'}


In [43]:
for i, row_to_compare in DOGC_sumarios.iterrows():
    for j, row_comparing in ASECORP_BBDD.iterrows():
        if set(row_to_compare['Tags']) & set(row_comparing['Tags']):
            DOGC_sumarios['Match_ASECORP_BBDD'][i].append (ASECORP_BBDD['Codigo'][j])
            print(str(set(row_to_compare['Tags']) & set(row_comparing['Tags'])) + ' ' + str(row_comparing['Codigo']))

{'Ley 16/2017'} LEY 16/2017 CAT
{'Ley 9/2019', 'Ley 16/2017'} LEY 9/2019 CAT
{'Ley 13/1989'} LEY 23/2002 CAT
{'Ley 3/2012'} DLE 3/2012 CAT
{'Ley 3/2012'} LEY 14/2015 CAT
{'Ley 3/2012'} LEY 3/2012 CAT
{'Ley 3/2012'} LEY 3/2012 ESP
{'Ley 5/2017'} LEY 5/2017 CAT
{'Ley 5/2017'} ORD EMC/155/2017 CAT
{'Ley 17/2015'} LEY 17/2015 ESP


In [44]:
DOGC_sumarios

,item_Title,item_urlHTML,item_urlPDF,item_fullText,item_urlXML,tipo_doc,fecha_doc,numero_doc,numero_control,entidad_emisora,CVE,numero_DOGC,fecha_DOGC,seccion_DOGC,Tags,Match_ASECORP_BBDD
0,"CORRECCIÓN DE ERRATAS en la Ley 9/2019, de 23 ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,Habiéndose detectado una errata en la Ley 9/20...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Corrección de erratas],[01/03/2021],[],[21061054],[Departamento de la Presidencia],[CVE-DOGC-B-21061054-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 9/2019, Ley 16/2017]","[LEY 16/2017 CAT, LEY 9/2019 CAT]"
1,"ORDEN VEH/49/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 159.1.a) del Estatuto de autonomía...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Orden],[01/03/2021],[VEH/0049/2021],[21061055],[Departamento de la Vicepresidencia y de Econo...,[CVE-DOGC-B-21061055-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 13/2008, Ley 19/2014, Ley 13/2005, Ley 39...",[LEY 23/2002 CAT]
2,"RESOLUCIÓN INT/570/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Departamento de Interior, mediante la Resol...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[26/02/2021],[INT/0570/2021],[21061058],[Departamento de Interior],[CVE-DOGC-B-21061058-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 39/2015, Resolución INT/3444/2020]",[]
3,"RESOLUCIÓN INT/571/2021, de 26 de febrero, de ...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"\nEl Departamento de Interior, mediante la Re...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[26/02/2021],[INT/0571/2021],[21061059],[Departamento de Interior],[CVE-DOGC-B-21061059-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 39/2015, Resolución INT/3439/2020]",[]
4,"DECRETO 12/2021, de 2 de marzo, de la Inspecci...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,El artículo 131.2 del Estatuto de autonomía de...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Decreto],[02/03/2021],[012/2021],[21061108],[Departamento de Educación],[CVE-DOGC-B-21061108-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 13/2008, Ley 26/2010, Ley 12/2009, Ley 39...",[]
5,"RESOLUCIÓN EDU/572/2021, de 25 de febrero, por...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"Mediante la Resolución EDU/772/2020, de 24 de ...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Resolución],[25/02/2021],[EDU/0572/2021],[21061060],[Departamento de Educación],[CVE-DOGC-B-21061060-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 26/2010, Resolución EDU/772/2020, Ley 39/...",[]
6,"ORDEN SLT/48/2021, de 1 de marzo, por la que s...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,"El Decreto 215/1990, de 30 de julio, de promoc...",[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Orden],[01/03/2021],[SLT/0048/2021],[21061061],[Departamento de Salud],[CVE-DOGC-B-21061061-2021],[8356],[04/03/2021],[Disposiciones],"[Orden de 15 de mayo de 1991, Orden PRE/194/20...",[]
7,"EDICTO de 25 de febrero de 2021, sobre un acue...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.gencat.cat/utilsEADOP/AppJa...,La Comisión Territorial de Urbanismo de El Cam...,[https://portaldogc.gencat.cat/utilsEADOP/AppJ...,[Edicto ],[25/02/2021],[],[21056014],[Departamento de Territorio y Sostenibilidad],[CVE-DOGC-B-21056014-2021],[8356],[04/03/2021],[Disposiciones],"[Ley 29/1998, Ley 2/2007, Ley 39/2015, Ley 3/2...","[DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 C..."
8,"ACUERDO GOV/22/2021, de 2 de marzo, por el que...",https://dogc.gencat.cat/es/document-del-dogc/?...,https://portaldogc.ge

# Genera Fichero EXCEL de resultados

In [45]:
DOGC_sumarios_final = DOGC_sumarios

In [46]:
DOGC_sumarios_final.dtypes

item_Title            object
item_urlHTML          object
item_urlPDF           object
item_fullText         object
item_urlXML           object
tipo_doc              object
fecha_doc             object
numero_doc            object
numero_control        object
entidad_emisora       object
CVE                   object
numero_DOGC           object
fecha_DOGC            object
seccion_DOGC          object
Tags                  object
Match_ASECORP_BBDD    object
dtype: object

In [47]:
## Cambia orden de columnas y elimina las no necesarias  
DOGC_sumarios_final.rename(columns={'CVE': 'Item_id', 'item_Title': 'Item_Title', 'item_urlPDF': 'PDF_Link', 'fecha_DOGC': 'Fecha_publicacion'}, inplace=True)
DOGC_sumarios_final = DOGC_sumarios[['Item_id','Item_Title','PDF_Link','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [48]:
## Crea función que convierte lista a string en todas las columnas de DOCM_sumarios
## para evitar en presentación final los caracteres [' '] propios de las listas
def list2Str(lst):
    if type(lst) is list: # apply conversion to list columns
        return", ".join(lst)
    else:
        return lst

In [49]:

DOGC_sumarios_final = DOGC_sumarios_final.apply(lambda x: [list2Str(i) for i in x])

In [50]:
## Generar hyperlink a artículo BOE en CSV "=HYPERLINK("https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf";"BOE-A-2021-1474")"
## https://www.boe.es/diario_boe/xml.php?id=## https://www.boe.es/boe/dias/2021/02/02/pdfs/BOE-A-2021-1474.pdf

DOGC_sumarios_final_CSV = DOGC_sumarios_final

In [51]:
DOGC_sumarios_final_CSV['item_id'] = '=HIPERVINCULO(' + '"' + DOGC_sumarios_final_CSV['PDF_Link'] + '";' \
                                                        + '"' + DOGC_sumarios_final_CSV['Item_id'] + '")'      

In [52]:
DOGC_sumarios_final_CSV['item_id'][0]

'=HIPERVINCULO("https://portaldogc.gencat.cat/utilsEADOP/AppJava/PdfProviderServlet?documentId=894810&type=01&language=es_ES";"CVE-DOGC-B-21061054-2021")'

In [53]:
# Elimina columna PDF_Link
DOGC_sumarios_final_CSV = DOGC_sumarios_final_CSV[['Item_id','Item_Title','Fecha_publicacion','Tags','Match_ASECORP_BBDD']]

In [54]:
DOGC_sumarios_final_CSV

,Item_id,Item_Title,Fecha_publicacion,Tags,Match_ASECORP_BBDD
0,CVE-DOGC-B-21061054-2021,"CORRECCIÓN DE ERRATAS en la Ley 9/2019, de 23 ...",04/03/2021,"Ley 9/2019, Ley 16/2017","LEY 16/2017 CAT, LEY 9/2019 CAT"
1,CVE-DOGC-B-21061055-2021,"ORDEN VEH/49/2021, de 1 de marzo, por la que s...",04/03/2021,"Ley 13/2008, Ley 19/2014, Ley 13/2005, Ley 39/...",LEY 23/2002 CAT
2,CVE-DOGC-B-21061058-2021,"RESOLUCIÓN INT/570/2021, de 26 de febrero, de ...",04/03/2021,"Ley 39/2015, Resolución INT/3444/2020",
3,CVE-DOGC-B-21061059-2021,"RESOLUCIÓN INT/571/2021, de 26 de febrero, de ...",04/03/2021,"Ley 39/2015, Resolución INT/3439/2020",
4,CVE-DOGC-B-21061108-2021,"DECRETO 12/2021, de 2 de marzo, de la Inspecci...",04/03/2021,"Ley 13/2008, Ley 26/2010, Ley 12/2009, Ley 39/...",
5,CVE-DOGC-B-21061060-2021,"RESOLUCIÓN EDU/572/2021, de 25 de febrero, por...",04/03/2021,"Ley 26/2010, Resolución EDU/772/2020, Ley 39/2015",
6,CVE-DOGC-B-21061061-2021,"ORDEN SLT/48/2021, de 1 de marzo, por la que s...",04/03/2021,"Orden de 15 de mayo de 1991, Orden PRE/194/202...",
7,CVE-DOGC-B-21056014-2021,"EDICTO de 25 de febrero de 2021, sobre un acue...",04/03/2021,"Ley 29/1998, Ley 2/2007, Ley 39/2015, Ley 3/2012","DLE 3/2012 CAT, LEY 14/2015 CAT, LEY 3/2012 CA..."
8,CVE-DOGC-B-21062071-2021,"ACUERDO GOV/22/2021, de 2 de marzo, por el que...",04/03/2021,"Ley 13/2008, Ley 5/2017, Ley 4/2008","LEY 5/2017 CAT, ORD EMC/155/2017 CAT"
9,CVE-DOGC-B-21055047-2021,"RESOLUCIÓN CLT/497/2021, de 16 de febrero, por...",04/03/2021,"Orden CLT/47/2014, Resolución CLT/198/2021, Re...",


In [55]:
DOGC_sumarios_final_CSV.to_csv("./ASECORP/Resultados_Matching_DOGC_" + today.strftime("%Y%m%d") + ".csv" ,index=False) 